
# 1. Define Paths and Mappings


In [0]:
from pyspark.sql.functions import col, lit, sha2, concat_ws, current_timestamp, coalesce, trim, to_timestamp, year
from delta.tables import DeltaTable
import datetime

# --- 1. Define Paths and Mappings ---

# Source Path (Input)
SOURCE_PATH = "abfss://project@scrgvkrmade.dfs.core.windows.net/bronze/ResellerSales/HumanResources.Employee/"
TARGET_PATH = "abfss://project@scrgvkrmade.dfs.core.windows.net/silver/dim/dim_Employee/"

# Define the Primary Key column(s) as a comma-separated string
PK_RAW = "CurrencyAlternateKey,CurrencyName" 

# --- PARSE COMPOSITE KEY ---
PK_COLS = [c.strip() for c in PK_RAW.split(",") if c.strip()]
print(f"Using Composite Primary Key for MERGE: {PK_COLS}")

# Columns needed for the Gold table
CURRENCY_MAPPING = [
    ("CurrencyCode", "CurrencyAlternateKey"), # Source column, Target column
    ("Name", "CurrencyName"),
    ("ModifiedDate", "ModifiedDate")
]

# Configure Authentication (use the DFS endpoint)
storage_account_name = "scrgvkrmade"
account_key = "E4VB7pXWFXttUWbbSBPY35/Dvsw6Fs6XgIWLTj3lCS6v/jCEow9Uxs+r6Usobhenv14UdWEzb+R8+AStNyS0dg=="
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    account_key
)
print("Configured Spark authentication.")




# 2. Read, Clean, and Prepare Source Data (df_source) 

In [0]:

# --- 2. Read, Clean, and Prepare Source Data (df_source) ---
print(f"\nReading Currency data from: {SOURCE_PATH}")
#df_source = spark.read.format("parquet").load(SOURCE_PATH)
df_source = (spark.read
              .option("mergeSchema", "true")
              .option("recursiveFileLookup", "true")
              .parquet(SOURCE_PATH))
print("Read OK. Rows:", df_source.count(), "Columns:", df_source.columns)
display(df_source.limit(5))

# Select and Rename columns using the manual mapping
selected_expr = [col(src).alias(tgt) for src, tgt in CURRENCY_MAPPING if src in df_source.columns]
df_source = df_source.select(*selected_expr)

# --- 2a. Data Cleaning and Deduplication ---

print("Applying simple data cleaning transformations...")

# 1. Standardize and Clean String Columns (trim only)
df_source = df_source.withColumn(
    "CurrencyAlternateKey",
    trim(col("CurrencyAlternateKey")) 
).withColumn(
    "CurrencyName",
    trim(col("CurrencyName"))
)

# 2. Handle Nulls 
df_source = df_source.withColumn(
    "CurrencyAlternateKey",
    coalesce(col("CurrencyAlternateKey"), lit("N/A")) # Ensure key is not null
).withColumn(
    "CurrencyName",
    coalesce(col("CurrencyName"), lit("Unknown Currency")) # Ensure name is not null
)

# 3. Type Casting
df_source = df_source.withColumn(
    "ModifiedDate",
    to_timestamp(col("ModifiedDate")) # Cast ModifiedDate to a proper Timestamp type
)

# 4. Check for duplicates and keep one (Deduplication)
initial_count = df_source.count()
df_source = df_source.dropDuplicates(PK_COLS)
final_count = df_source.count()

print(f"Deduplication complete. Removed {initial_count - final_count} duplicates based on {PK_COLS}.")

# --- 2b. Add Audit and Hash Columns ---

# Add Audit Columns (ALWAYS create)
df_source = df_source.withColumn("__ingest_ts", current_timestamp())
df_source = df_source.withColumn("__source_path", lit(SOURCE_PATH)) 
df_source = df_source.withColumn("__target_path", lit(TARGET_PATH)) 
df_source = df_source.withColumn("__batch_id", lit("Batch-" + datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')))

# Create a hash for change detection
hash_cols = ["CurrencyAlternateKey", "CurrencyName"]
df_source = df_source.withColumn(
    "__row_hash",
    sha2(concat_ws("||", *[col(c).cast("string") for c in hash_cols]), 256)
)

# Add required DimCurrency columns (placeholders)
df_source = df_source.withColumn("LoadTS", current_timestamp())
df_source = df_source.withColumn("IsCurrent", lit(True))
df_source = df_source.withColumn("CurrencyKey", lit(None).cast("long")) # Placeholder for surrogate key
df_source = df_source.withColumn("_year", year(col("ModifiedDate"))) # year

# 3. MERGE (Incremental Upsert)

In [0]:
# --- 3. Check and Create Target Table (if needed) ---
target_exists = DeltaTable.isDeltaTable(spark, TARGET_PATH)

# List of all final columns for the DimCurrency schema
# ALL_TARGET_COLS = [
#     #"CurrencyKey", 
#     "CurrencyAlternateKey", "CurrencyName", "ModifiedDate", "LoadTS", 
#     "__row_hash", "IsCurrent", "_year", "__ingest_ts", "__source_path", "__target_path", "__batch_id"
# ]
ALL_TARGET_COLS = [
    "CurrencyAlternateKey", "CurrencyName", "ModifiedDate", "LoadTS", 
    "__row_hash", "IsCurrent", "_year", "__ingest_ts", "__source_path", "__target_path", "__batch_id"
]


if not target_exists:
    print(f"Target table not found. Creating initial DimCurrency table at: {TARGET_PATH}")
    
    # Select only the required columns and write the initial table
    df_source.select(*[c for c in ALL_TARGET_COLS if c in df_source.columns]).write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true")\
        .partitionBy("_year")\
        .save(TARGET_PATH)
    print("Initial DimCurrency table created. Skipping merge.")

else:
    # --- 4. Perform MERGE (Incremental Upsert) ---
    print(f"\nTarget table exists. Performing MERGE (Incremental Load)...")
    
    dt_target = DeltaTable.forPath(spark, TARGET_PATH)
    
    # Define the condition to match an existing row (Primary Key/Alternate Key)
    join_cond = f"target.{PK_COLS} = source.{PK_COLS}"
    
    # Define the condition to detect a change (Row Hash)
    change_cond = "target.__row_hash != source.__row_hash"

    dt_target.alias("target").merge(
        df_source.alias("source"),
        join_cond
    ) \
    .whenMatchedUpdate(
        condition=change_cond,
        set = {
            # Update mutable data columns
            "CurrencyName": "source.CurrencyName",
            "ModifiedDate": "source.ModifiedDate",
            # Update audit columns on change
            "LoadTS": "source.LoadTS",
            "__row_hash": "source.__row_hash",
            "__ingest_ts": "source.__ingest_ts",
            "__batch_id": "source.__batch_id"
        }
    ) \
    .whenNotMatchedInsert(
        values = {
            # Insert all columns for new records
            #"CurrencyKey": "NULL",
            "CurrencyAlternateKey": "source.CurrencyAlternateKey",
            "CurrencyName": "source.CurrencyName",
            "ModifiedDate": "source.ModifiedDate",
            "LoadTS": "source.LoadTS",
            "__row_hash": "source.__row_hash",
            "IsCurrent": "source.IsCurrent",
            "_year": "source._year",            
            "__ingest_ts": "source.__ingest_ts",
            "__source_path": "source.__source_path",
            "__target_path": "source.__target_path",
            "__batch_id": "source.__batch_id"
        }
    ) \
    .execute()
    
    print("MERGE (Incremental Load) complete. ")

In [0]:
# KEY for ABFSS (dfs endpoint) - needed for writing Delta
spark.conf.set(
    "fs.azure.account.key.scrgvkrmade.dfs.core.windows.net",
    "E4VB7pXWFXttUWbbSBPY35/Dvsw6Fs6XgIWLTj3lCS6v/jCEow9Uxs+r6Usobhenv14UdWEzb+R8+AStNyS0dg=="
)

# Read from BLOB (works)
df = spark.read.parquet(
    "wasbs://project@scrgvkrmade.blob.core.windows.net/bronze/ResellerSales/Sales.Currency/*/part-*.parquet"
)

#display(df)
display(df.limit(5))

In [0]:
# read table root
df = spark.read.format("delta").load("wasbs://project@scrgvkrmade.blob.core.windows.net/silver/dim/dim_currency")

# read a specific partition (also allowed)
df_part = spark.read.format("delta").load(
    "wasbs://project@scrgvkrmade.blob.core.windows.net/silver/dim/dim_currency"
)
display(df_part)

In [0]:
from pyspark.sql.functions import (
    col, lit, sha2, concat_ws, current_timestamp, coalesce, trim, 
    to_timestamp, year
)
from delta.tables import DeltaTable
import datetime
import traceback
import sys

# --- 1. Define Paths and Mappings ---

# Source Path (Input)

# Source Path (Input)
SOURCE_PATH = "abfss://project@scrgvkrmade.dfs.core.windows.net/bronze/ResellerSales/Sales.Currency/"
TARGET_PATH = "abfss://project@scrgvkrmade.dfs.core.windows.net/silver/dim/dim_currency/"

# Define the Primary Key column(s) as a comma-separated string (Composite Key)
PK_RAW = "CurrencyAlternateKey,CurrencyName" 

# --- PARSE COMPOSITE KEY ---
PK_COLS = [c.strip() for c in PK_RAW.split(",") if c.strip()]
print(f"Using Composite Primary Key for MERGE: {PK_COLS}")
# ---------------------------

# Columns needed for the Gold DimCurrency table
# Mapping format: ("Source_Column_Name", "Target_Column_Name")
CURRENCY_MAPPING = [
    ("CurrencyCode", "CurrencyAlternateKey"), 
    ("Name", "CurrencyName"),
    ("ModifiedDate", "ModifiedDate")
]

# Configure Authentication (REPLACE WITH REAL KEY)
storage_account_name = "scrgvkrmade"
account_key = "E4VB7pXWFXttUWbbSBPY35/Dvsw6Fs6XgIWLTj3lCS6v/jCEow9Uxs+r6Usobhenv14UdWEzb+R8+AStNyS0dg=="
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    account_key
)
print("Configured Spark authentication.")

# ---------------------------------------------------------------------------------------
# --- 2. Read, Clean, and Prepare Source Data (df_source) ---
# ---------------------------------------------------------------------------------------

print(f"\nReading Currency data from: {SOURCE_PATH} (Multiple Parquet files)")
try:
    df_source = (spark.read
                  .option("mergeSchema", "true")
                  .option("recursiveFileLookup", "true")
                  .parquet(SOURCE_PATH))
    print(f"Read OK. Rows: {df_source.count()} Columns: {df_source.columns}")
except Exception:
    print("Failed to read multiple Parquet files. Check path and permissions.")
    traceback.print_exc(file=sys.stdout)
    raise

# Select and Rename columns using the manual mapping
selected_expr = [col(src).alias(tgt) for src, tgt in CURRENCY_MAPPING if src in df_source.columns]
df_source = df_source.select(*selected_expr)

# --- 2a. Data Cleaning and Deduplication ---

print("Applying simple data cleaning transformations...")

# 1. Standardize and Clean String Columns (trim only)
df_source = df_source.withColumn("CurrencyAlternateKey", trim(col("CurrencyAlternateKey")))
df_source = df_source.withColumn("CurrencyName", trim(col("CurrencyName")))

# 2. Handle Nulls 
df_source = df_source.withColumn(
    "CurrencyAlternateKey",
    coalesce(col("CurrencyAlternateKey"), lit("N/A"))
).withColumn(
    "CurrencyName",
    coalesce(col("CurrencyName"), lit("Unknown Currency"))
)

# 3. Type Casting (Crucial for year extraction)
df_source = df_source.withColumn(
    "ModifiedDate",
    to_timestamp(col("ModifiedDate"))
)

# 4. Deduplication
initial_count = df_source.count()
df_source = df_source.dropDuplicates(PK_COLS)
final_count = df_source.count()
print(f"Deduplication complete. Removed {initial_count - final_count} duplicates based on {PK_COLS}.")


# --- 2b. Add Audit and Hash Columns ---

# Add Audit Columns
df_source = df_source.withColumn("__ingest_ts", current_timestamp())
df_source = df_source.withColumn("__source_path", lit(SOURCE_PATH)) 
df_source = df_source.withColumn("__target_path", lit(TARGET_PATH)) 
df_source = df_source.withColumn("__batch_id", lit("Batch-" + datetime.datetime.utcnow().strftime('%Y%m%d%H%M%S')))

# Create a hash for change detection
hash_cols = ["CurrencyAlternateKey", "CurrencyName"]
df_source = df_source.withColumn(
    "__row_hash",
    sha2(concat_ws("||", *[coalesce(col(c).cast("string"), lit("")) for c in hash_cols]), 256)
)

# Add required DimCurrency columns (placeholders)
df_source = df_source.withColumn("LoadTS", current_timestamp())
df_source = df_source.withColumn("IsCurrent", lit(True))
df_source = df_source.withColumn("CurrencyKey", lit(None).cast("long")) # Placeholder for surrogate key

# ADD _year for Partitioning (Using the imported 'year' function)
df_source = df_source.withColumn("_year", year(col("ModifiedDate")))


# ---------------------------------------------------------------------------------------
# --- 3. Check and Create Target Table (if needed) and MERGE ---
# ---------------------------------------------------------------------------------------

target_exists = DeltaTable.isDeltaTable(spark, TARGET_PATH)

# List of all final columns for the DimCurrency schema
ALL_TARGET_COLS = [
     "CurrencyAlternateKey", "CurrencyName", "ModifiedDate", "LoadTS", 
    "__row_hash", "IsCurrent", "_year", "__ingest_ts", "__source_path", "__target_path", "__batch_id"
]


if not target_exists:
    print(f"Target table not found. Creating initial Delta table at: {TARGET_PATH}")
    
    df_source.select(*[c for c in ALL_TARGET_COLS if c in df_source.columns]).write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .partitionBy("_year") \
        .save(TARGET_PATH)
    print("Initial write completed.")

else:
    # --- Perform MERGE (Incremental Upsert) ---
    print(f"\nTarget table exists. Performing MERGE (Incremental Load)...")
    
    dt_target = DeltaTable.forPath(spark, TARGET_PATH)
    
    # Define the condition to match an existing row (Composite Primary Key)
    # This correctly generates the SQL join string: "target.Col1 = source.Col1 AND target.Col2 = source.Col2"
    join_cond = " AND ".join([f"target.{c} = source.{c}" for c in PK_COLS])
    
    # Define the condition to detect a change (Row Hash)
    change_cond = "target.__row_hash != source.__row_hash"
    
    print(f"MERGE Join Condition: {join_cond}")

    dt_target.alias("target").merge(
        df_source.alias("source"),
        join_cond
    ) \
    .whenMatchedUpdate(
        condition=change_cond,
        set = {
            # Update mutable data and audit columns
            "CurrencyName": "source.CurrencyName",
            "ModifiedDate": "source.ModifiedDate",
            "LoadTS": "source.LoadTS",
            "__row_hash": "source.__row_hash",
            "__ingest_ts": "source.__ingest_ts",
            "__batch_id": "source.__batch_id"
        }
    ) \
    .whenNotMatchedInsert(
        # Insert all columns for new records
        #values = {c: f"source.{c}" for c in ALL_TARGET_COLS if c in df_source.columns}
        values = {
            # Insert all columns for new records
            #"CurrencyKey": "NULL",
            "CurrencyAlternateKey": "source.CurrencyAlternateKey",
            "CurrencyName": "source.CurrencyName",
            "ModifiedDate": "source.ModifiedDate",
            "LoadTS": "source.LoadTS",
            "__row_hash": "source.__row_hash",
            "IsCurrent": "source.IsCurrent",
            "_year": "source._year",            
            "__ingest_ts": "source.__ingest_ts",
            "__source_path": "source.__source_path",
            "__target_path": "source.__target_path",
            "__batch_id": "source.__batch_id"
        }
    ) \
    .execute()
    
    print("MERGE (Incremental Load) complete. ✅")

# ---------------------------------------------------------------------------------------
# --- 4. Validation (Optional) ---
# ---------------------------------------------------------------------------------------

print("\nValidating output by reading back Target_path:", TARGET_PATH)
try:
    tgt = spark.read.format("delta").load(TARGET_PATH)
    print("Rows in Gold:", tgt.count())
except Exception:
    print("Could not read back Delta target.")
    traceback.print_exc(file=sys.stdout)
    
print("Done.")